<a href="https://colab.research.google.com/github/zoikorda/self-driving-cars/blob/main/SoftActorCritic-HER_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install environment and agent
!pip install highway-env
!pip install stable-baselines3
!pip install sb3-contrib

# Environment
import gym
import highway_env

# Agent
from stable_baselines3 import HerReplayBuffer, SAC
from sb3_contrib import TQC

In [ ]:
env = gym.make("parking-v0")
her_kwargs = dict(n_sampled_goal=4, goal_selection_strategy='future', online_sampling=True, max_episode_length=100)
# You can replace TQC with SAC agent
model = SAC('MultiInputPolicy', env, replay_buffer_class=HerReplayBuffer,
            replay_buffer_kwargs=her_kwargs, verbose=1, buffer_size=int(1e6),
            learning_rate=1e-3,
            gamma=0.95, batch_size=1024, tau=0.05,
            policy_kwargs=dict(net_arch=[512, 512, 512]))
model.learn(int(5e4))


In [ ]:
!pip install gym pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg

from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
from gym.wrappers import Monitor
from pathlib import Path
import base64
from tqdm.notebook import trange

display = Display(visible=0, size=(1400, 900))
display.start()

def show_video():
    html = []
    for mp4 in Path("video").glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append('''<video alt="{}" autoplay 
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>'''.format(mp4, video_b64.decode('ascii')))
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


In [ ]:
obs = env.reset()

# Evaluate the agent
rewards = []
episode_reward = 0
for _ in range(1000):
  action, _ = model.predict(obs)
  obs, reward, done, info = env.step(action)
  episode_reward += reward
  if done or info.get('is_success', False):
    print("Reward:", episode_reward, "Success?", info.get('is_success', False))
    rewards.append(episode_reward)
    episode_reward = 0.0
    obs = env.reset()

In [ ]:
env = gym.make("parking-v0")
env = Monitor(env, './video', force=True, video_callable=lambda episode: True)
rewards = []
for episode in trange(3, desc="Test episodes"):
    obs, done = env.reset(), False
    env.unwrapped.automatic_rendering_callback = env.video_recorder.capture_frame
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        rewards.append(reward)
env.close()
show_video()

In [ ]:
print(rewards)

import matplotlib.pyplot as plt
import numpy as np

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(rewards)+1), rewards)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()